# Pendulum: Connected insoles for fall risk prediction

## Workflow steps

1. Problem Definition
2. Import Train and Test Set
3. Explore Data and Generate New Features
4. Feature Standardization
5. Model Training and Prediction
6. Prediction on the Test Set
7. Export the Predicted Results

## 1. Problem Definition and Goal: 
Analyse daily activity (steps, balance, gait, ...) and provide reccomendation exercises based on fall risk detection/prediction

## 2. Import train and test set - ATM Import Datasets from research papers


In [2]:
# Load libraries for data analysis 
import pandas as pd
import numpy as np
import random
#from pandas.tools.plotting import scatter_matrix
# Load libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Load libraries for machine learning
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

Datasets from SisFall: A Fall and Movement Dataset (Sucerquia et al., Sensors 2017) <br>
Features: amplitude, orientation angle, statistical moments, critical phase time, and area under the curve <br>
 This database consists of 4,510 files, each file with a single activity. <br>
 SA: Adults subjects between 19 and 30 years old <br>
  SE: Elderly people between 60 and 75 years old <br>

Elderly people only simulated ADLs except SE06, who is an expert in Judo that also simulated falls. Elderly people did not 
perform activities D06, D13, D18, and D19 due to recommendations of a physician specialized in sports. Additionally, some 
elderly people did not perform some activities due to personal impairments (or medical recommendation). <br>

Each file contains nine columns and a different number of rows depending on the test length. <br>

1st column is the acceleration data in the X axis measured by the sensor ADXL345.  <br>
2nd column is the acceleration data in the Y axis measured by the sensor ADXL345. <br>
3rd column is the acceleration data in the Z axis measured by the sensor ADXL345. <br>

4th column is the rotation data in the X axis measured by the sensor ITG3200. <br>
5th column is the rotation data in the Y axis measured by the sensor ITG3200. <br>
6th column is the rotation data in the Z axis measured by the sensor ITG3200. <br>

7th column is the acceleration data in the X axis measured by the sensor MMA8451Q. <br>
8th column is the acceleration data in the Y axis measured by the sensor MMA8451Q. <br>
9th column is the acceleration data in the Z axis measured by the sensor MMA8451Q. <br>

Data are in bits with the following characteristics:

ADXL345:
Resolution: 13 bits
Range: +-16g

ITG3200
Resolution: 16 bits
Range: +-2000°/s

MMA8451Q:
Resolution: 14 bits
Range: +-8g


In order to convert the acceleration data (AD) given in bits into gravity, use this equation: <br>

Acceleration [g]: [(2*Range)/(2^Resolution)]*AD 

In order to convert the rotation data (RD) given in bits into angular velocity, use this equation:

Angular velocity [°/s]: [(2*Range)/(2^Resolution)]*RD





-+--------+----------+
| Code | Activity                                                                                 | Trials | Duration |
+------+------------------------------------------------------------------------------------------+--------+----------+
| D01  | Walking slowly                                                                           | 1      | 100s     |
| D02  | Walking quickly                                                                          | 1      | 100s     |
| D03  | Jogging slowly                                                                           | 1      | 100s     |
| D04  | Jogging quickly                                                                          | 1      | 100s     |
| D05  | Walking upstairs and downstairs slowly                                                   | 5      | 25s      |
| D06  | Walking upstairs and downstairs quickly                                                  | 5      | 25s      |
| D07  | Slowly sit in a half height chair, wait a moment, and up slowly                          | 5      | 12s      |
| D08  | Quickly sit in a half height chair, wait a moment, and up quickly                        | 5      | 12s      |
| D09  | Slowly sit in a low height chair, wait a moment, and up slowly                           | 5      | 12s      |
| D10  | Quickly sit in a low height chair, wait a moment, and up quickly                         | 5      | 12s      |
| D11  | Sitting a moment, trying to get up, and collapse into a chair                            | 5      | 12s      |
| D12  | Sitting a moment, lying slowly, wait a moment, and sit again                             | 5      | 12s      |
| D13  | Sitting a moment, lying quickly, wait a moment, and sit again                            | 5      | 12s      |
| D14  | Being on oneís back change to lateral position, wait a moment, and change to oneís back  | 5      | 12s      |
| D15  | Standing, slowly bending at knees, and getting up                                        | 5      | 12s      |
| D16  | Standing, slowly bending without bending knees, and getting up                           | 5      | 12s      |
| D17  | Standing, get into a car, remain seated and get out of the car                           | 5      | 25s      |
| D18  | Stumble while walking                                                                    | 5      | 12s      |
| D19  | Gently jump without falling (trying to reach a high object)                              | 5      | 12s      |
+------+------------------------------------------------------------------------------------------+--------+----------+

Falls:

+------+------------------------------------------------------------------------------------------+--------+----------+
| Code | Activity                                                                                 | Trials | Duration |
+------+------------------------------------------------------------------------------------------+--------+----------+
| F01  | Fall forward while walking caused by a slip                                              | 5      | 15s      |
| F02  | Fall backward while walking caused by a slip                                             | 5      | 15s      |
| F03  | Lateral fall while walking caused by a slip                                              | 5      | 15s      |
| F04  | Fall forward while walking caused by a trip                                              | 5      | 15s      |
| F05  | Fall forward while jogging caused by a trip                                              | 5      | 15s      |
| F06  | Vertical fall while walking caused by fainting                                           | 5      | 15s      |
| F07  | Fall while walking, with use of hands in a table to dampen fall, caused by fainting      | 5      | 15s      |
| F08  | Fall forward when trying to get up                                                       | 5      | 15s      |
| F09  | Lateral fall when trying to get up                                                       | 5      | 15s      |
| F10  | Fall forward when trying to sit down                                                     | 5      | 15s      |
| F11  | Fall backward when trying to sit down                                                    | 5      | 15s      |
| F12  | Lateral fall when trying to sit down                                                     | 5      | 15s      |
| F13  | Fall forward while sitting, caused by fainting or falling asleep                         | 5      | 15s      |
| F14  | Fall backward while sitting, caused by fainting or falling asleep                        | 5      | 15s      |
| F15  | Lateral fall while sitting, caused by fainting or falling asleep                         | 5      | 15s      |
+------+------------------------------------------------------------------------------------------+--------+----------+

In [20]:
 # Activity dataset - each file is for one person, a specific activity and a given trial (ROx) 
    # Walking slowly
activity_ds_S1 = pd.read_csv('SisFall_dataset-3/SA01/D01_SA01_R01.txt', sep=',', names = ['acceleration1_x','acceleration1_y','acceleration1_z','rotation1_x','rotation1_y','rotation1_z','acceleration2_x','accelaration2_y','acceleration2_z'])
activity_ds_S1.head()


,acceleration1_x,acceleration1_y,acceleration1_z,rotation1_x,rotation1_y,rotation1_z,acceleration2_x,accelaration2_y,acceleration2_z
0,17,-179,-99,-18,-504,-352,76,-697,-279;
1,15,-174,-90,-53,-568,-306,48,-675,-254;
2,1,-176,-81,-84,-613,-271,-2,-668,-221;
3,-10,-180,-77,-104,-647,-227,-34,-697,-175;
4,-21,-191,-63,-128,-675,-191,-74,-741,-133;


## Feature generation (as calculated in Table 4)

In [ ]:
##### Amplitude features #####

sum_vector_magnitude = round(np.sqrt(activity_ds_S1['acceleration1_x'].pow(2)+activity_ds_S1['acceleration1_y'].pow(2)+activity_ds_S1['acceleration1_z'].pow(2)),2)
sum_vector_magnitude.head()

In [26]:
# Horizontal Plane
sum_vector_magnitude_hp = round(np.sqrt(activity_ds_S1['acceleration1_x'].pow(2)+activity_ds_S1['acceleration1_z'].pow(2)),2)
sum_vector_magnitude_hp.head()


0    100.45
1     91.24
2     81.01
3     77.65
4     66.41
dtype: float64

In [ ]:
# Maximum peak-to-peak acceleration amplitude



In [36]:
##### Orientation #####

# Orientation change in horizontal plane - assuming window size of 300ms

window_size = 300 
or_change_hp = ((activity_ds_S1['acceleration1_x']- window_size).mean()) * (activity_ds_S1['acceleration1_x'].mean())
or_change_hp

-2688.9342903942043

In [ ]:
#### Time ####
# Acceleration change rate (jerk)

# assuming t is the time of the activity [s]? 
time = 10

jerk = (activity_ds_S1['acceleration1_x'] - activity_ds_S1['acceleration1_x']- window_size) / time

In [18]:
activity_ds_S1_D5 = pd.read_csv('SisFall_dataset-3/SA01/D05_SA01_R01.txt', sep=',',header=0)

activity_ds_S1_D5.size

44991

In [ ]:
## Amplitude features

sum_vector_magnitude = np.sqrt()